In [1]:
from pathlib import Path
from typing import List, Tuple
import os

import pandas as pd
from bert_score import score
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams
from langchain_openai import ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_qdrant import QdrantVectorStore
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.retrievers import MultiQueryRetriever
from langchain.chains import HypotheticalDocumentEmbedder
from langchain_core.documents import Document
import pdfplumber
from pdf2image import convert_from_path
import pytesseract

from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [18]:
PROXYAPI_BASE_URL = os.getenv("PROXYAPI_BASE_URL")
PROXYAPI_KEY = os.getenv("PROXYAPI_KEY")
QDRANT_URL = os.getenv("QDRANT_URL")
QDRANT_API_KEY = os.getenv("QDRANT_API_KEY")
PDF_PATH = r"..\data\Руководство по насосам шламовым 4 DY -AHF.pdf"

In [4]:
VEC_SIZE = 1024

### 0. Создание коллекции в Qdrant

Создадим коллекцию Qdrant для последующей загрузки документов

In [ ]:
client = QdrantClient(url=QDRANT_URL, api_key=QDRANT_API_KEY)

collection_name = "manual"

if not client.collection_exists(collection_name=collection_name):
    client.create_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=VEC_SIZE, distance=Distance.COSINE),
)

### 1. Извлечение текста из документа

Так как исходный PDF-файл не оцифрован, используем технологии OCR для конвертации содержимого в текст. Будем применять OCR для каждой страницы отдельно, чтобы впоследствии сохранить номер страницы в Qdrant

In [6]:
def extract_pdf_pages(pdf_path: str,
                      dpi: int = 300,
                      ocr_lang: str = "eng+rus") -> List[Tuple[str, int]]:
    pdf_path = Path(pdf_path).expanduser().resolve()
    results: List[Tuple[str, int]] = []

    with pdfplumber.open(pdf_path) as pdf:
        for page_num, page in enumerate(pdf.pages, start=1):
            img = convert_from_path(
                pdf_path,
                dpi=dpi,
                first_page=page_num,
                last_page=page_num,
                fmt="png"
            )[0]

            text = pytesseract.image_to_string(img, lang=ocr_lang).strip()

            results.append((text, page_num))

    return results

pages = extract_pdf_pages(PDF_PATH, dpi=400)
print(f"Parsed {len(pages)} pages.")

Parsed 29 pages.


Сохраним текстовое содержимое исходного файла в txt-формате

In [7]:
with open("..\data\pages.txt", "w", encoding="utf-8") as f:
    for page in pages:
        f.write(page[0] + "\n" + 50 * "=" + "\n")

Не у всех элементов исходного файла после применения `pdf2image + pytesseract` удалось корректно сохранить внутреннюю структуру. В первую очередь это касалось таблиц (например, таблица со стр. 15). Так как в таблицах могут содержаться данные, необходимые для ответов на вопросы пользователей, необходимо было добавить их содержимое в финальное текстовое представление исходного файла.

Поскольку количество таблиц в документе небольшое (менее 10), было принято решение обработать их полуручным способом. Содержимое таблиц конвертировалось в csv-файл с помощью сторонних сервисов, затем содержимое документов переносилось в финальный документ `pages_processed.txt`.

### 2. Chunking и векторизация текстов

Разделим текст, полученный на предыдущем этапе, на чанки

In [8]:
def split_pages_into_chunks(
    pages: List[Tuple[str, int]],
    chunk_size: int = 800,
    chunk_overlap: int = 100,
    separators: List[str] | None = None,
) -> List[Tuple[str, int]]:
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=separators,
        add_start_index=False,
    )

    page_chunks: List[Tuple[str, int]] = []
    for page_text, page_no in pages:
        for chunk in splitter.split_text(page_text):
            page_chunks.append((chunk, page_text, page_no))

    return page_chunks

Обернем полученные чанки в `langchain_core.documents.Document()`, в метаданных для каждого чанка сохраним также номер страницы, из которой был взят соответствующий чанк

In [9]:
with open("../data/pages_processed.txt", "r", encoding="utf-8") as file:
    content = file.read()
pages_processed = content.split(50 * "=")
pages_processed = [(page_text.strip(), page_no + 1) for page_no, page_text in enumerate(pages_processed)]

In [10]:
chunks = split_pages_into_chunks(pages_processed, chunk_size=400, chunk_overlap=100)
print(len(chunks))

docs = [
    Document(
        page_content=f"passage: {chunk}",
        metadata={"page": page_no, "page_text": page_text}
    )
    for chunk, page_text, page_no in chunks
]

262


Сформируем эмбеддинги чанков с помощью `intfloat/multilingual-e5-large`

In [11]:
embeddings = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large", encode_kwargs={"normalize_embeddings": True})

In [14]:
qdrant = QdrantVectorStore.from_documents(
    documents=docs,
    embedding=embeddings,
    url=QDRANT_URL,
    api_key=QDRANT_API_KEY,
    prefer_grpc=True,
    collection_name="manual",
)

In [15]:
base_retriever = qdrant.as_retriever(search_kwargs={'k': 3})

docs = base_retriever.invoke("query: Введите номер телефона сервисной линии для клиентов в Финляндии.")

for doc in docs:
    print(f"Страница номер {doc.metadata['page']}:\n\n{doc.page_content}")

Страница номер 1:

passage: SPARES & SUPPORT HELPLINES:

SALES OFFICE | TELEPHONE
Benelux | +31 77 3272 840
Czech Republic | +420 543 518 300
England | +44 1706 814251
Finland | +358 3 877 350
France | +33 4 72 8106 29
Germany | +49 7131 640090
Hungary | +36 34 314 794
Italy | +39 02 9244 321
Poland | +48 632 8473
Romania | +40 259 465344
Russia | +70 95 775 08 67
Sweden | +46 920 870 77
Ukraine | +380 56 778 31 87
Страница номер 27:

passage: размещения заказа направить K клиенту
квалифицированных специалистов — для ремонта
оборудования.
Страница номер 27:

passage: Контракты на сервисное обслуживание были заключены с
рядом крупных заказчиков, там, где локальные ресурсы
сервисного обслуживания недоступны. По вопросу
заключения контракта на техническое обслуживание Вам
следует связаться с сервисным центром в Тодмордене.


### 3. Настройка RAG-пайплайна

В качестве LLM будем использовать `gpt-4o-mini`. Для этого будем использовать сервис [proxyapi](https://proxyapi.ru)

In [19]:
llm = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0,
    api_key=PROXYAPI_KEY,
    base_url=PROXYAPI_BASE_URL,
    top_p=0.8,
    max_completion_tokens=512,
    seed=42,
)

Составим промпт к LLM, опираясь на требования к разрабатываемой системе

In [20]:
RAG_TEMPLATE = """Вы — интеллектуальный помощник, специализирующийся на технической документации по насосам.

Инструкция  
1. Отвечайте на любые вопросы пользователя, опираясь исключительно на сведения, находящиеся внутри тега <context>.  
2. Если необходимой информации нет в документе, ответьте ровно: **Не знаю**.  
3. Если вопрос не относится к руководству о насосе, ответьте ровно: **Вопрос не относится к руководству о насосе**.  
4. В остальных случаях верните:  
   • краткий текстовый ответ (1-3 предложения);  
   • ссылки на использованные фрагменты — укажите цитату и/или номер страницы в круглых скобках;  
   • при необходимости ссылку на изображение или номер рисунка, если это сделает ответ понятнее.

Формат ответа  
Текст ответа (стр. N)  
[Изображение: название/номер рисунка]   ← опционально

<context>
{context}
</context>

Вопрос пользователя:
{input}

Ответ:

"""

rag_prompt = ChatPromptTemplate.from_template(RAG_TEMPLATE)

Так как запросы пользователей, как правило, являются короткими вопросами, поиск релевантных документов будем осуществлять по чанкам, хранящимся в `Qdrant`. Однако в качестве контекста для LLM будем подавать полный текст страниц, на которых находятся наиболее релевантные чанки. Это позволит дать LLM наиболее полный контекст для ответа на вопрос пользователя. Как показали эксперименты, благодаря такому подходу удается повысить точность ответов 

In [21]:
def format_docs(docs):
    '''Convert loaded documents into strings by concatenating their content and ignoring metadata'''
    return "\n\n".join(f"Страница номер {doc.metadata['page']}: {doc.metadata['page_text']}"  for doc in docs)

Для улучшения качества ответов используем следующие модификации базового RAG:
- HyDE для подмены запроса пользователя ответом на основе гипотетического документа
- Multi-Query для генерации запросов с разных точек зрения

In [22]:
hyde_embeddings = HypotheticalDocumentEmbedder.from_llm(
    llm=llm,
    base_embeddings=embeddings,
    prompt_key="web_search",
)

qdrant_hyde = QdrantVectorStore(
    client=qdrant.client,
    collection_name="manual",
    embedding=hyde_embeddings,
)

hyde_retriever = qdrant_hyde.as_retriever(search_kwargs={"k": 4})

multi_query_retriever = MultiQueryRetriever.from_llm(
    retriever=hyde_retriever,
    llm=llm,
    include_original=True,
)

In [23]:
retrieval_chain = ({"context": multi_query_retriever | format_docs, "input": RunnablePassthrough()}
    | rag_prompt
    | llm
    | StrOutputParser()
)

Протестируем построенную систему на контрольных вопросах

In [24]:
queries_answers = {
    "Какой объём смазки (л) требуется для подшипниковой рамы DY?": "Для подшипниковой рамы типа DY предусмотрен объём смазки 0.5 литра.",
    "Укажите кинематическую вязкость рекомендованного масла при 40 °C для роликовых подшипников.": "Для роликовых подшипников рекомендовано масло с кинематической вязкостью 150 mm²/s при 40 °C.",
    "Назовите заводскую смазку, рекомендованную для камеры центробежного уплотнения.": "Для смазки камеры центробежного уплотнения следует применять заводскую смазку FUCHS CENTARUS 4.",
    "Введите номер телефона сервисной линии для клиентов в Финляндии.": "Для клиентов в Финляндии действует сервисная линия по телефону +358 3 877 350.",
    "Какова ширина ремня (мм), указанная для диапазона шкивов 170–224 мм (профиль SPB)?": "Для диапазона шкивов 170–224 мм (профиль SPB) ширину ремня составляет 50 мм.",
    "Перечислите три основных требования техники безопасности при подъёме насоса с вертикальным приводом (CV).": "1. Поручить операцию компетентному работнику, знакомым с нормами безопасности и практикой строповки. 2. Использовать двухкольцевой строп, надетый удавкой, чтобы исключить проскальзывание корпуса. 3. Для избежания опрокидывания необходимо установить подпорку под смачиваемой частью насоса.",
    "Опишите процедуру проверки уровня масла в подшипниковом узле.": "Проверка уровня масла проводится на остановленном и выставленном по горизонту насосе: извлеките щуп из подшипникового узла, удалите капли, вновь опустите до упора и убедитесь, что масляный след находится между нижней и верхней отметками на щупе. При отклонении долейте или слейте масло до нормы.",
    "Какой класс NLGI указан для пластичной смазки камеры центробежного уплотнения?": "Класс NLGI для пластичной смазки камеры центробежного уплотнения указан как 4.",
    "Какой индекс вязкости DIN‑ISO 2909 указан для рекомендуемого масла подшипников?": "Рекомендуемое подшипниковое масло характеризуется индексом вязкости 91 по DIN-ISO 2909.",
    "На сколько литров объём смазки рамы FFY превышает объём рамы BY?": "Объём смазки рамы FFY = 3.0 л, тогда как для рамы BY = 0.13 л. Таким образом, FFY требует на 2.87 л больше смазочного материала.",
    "Согласно схеме LD002‑RUS WP8, какой тип стропа рекомендован, чтобы предотвратить проскальзывание насоса при подъёме?": "Согласно схеме LD002-RUS WP8, во избежание проскальзывания насоса при подъёме следует применять двухкольцевой строп, надеваемый удавкой на корпус.",
    "Сформируйте краткую инструкцию «Ежедневное ТО + подъём» (≤ 100 слов) и приложите соответствующую иллюстрацию.": "Перед запуском остановите насос, убедитесь, что корпус установлен горизонтально, и с помощью щупа проверьте, чтобы уровень масла в подшипниковой раме оставался между метками. Осмотрите узлы на предмет утечек, износа и механических повреждений. При необходимости подтяните крепёж. Для подъёма насоса CV привлеките квалифицированного человека, примените двухкольцевой строп-удавку с мягкими защитными лентами; выравняйте длину ветвей, создайте плавное натяжение и поднимайте без рывков, удерживая корпус в вертикальном положении. (см. рис. LD002-RUS WP8)"
}

df = pd.DataFrame.from_records(list(queries_answers.items()), columns=["queries", "reference"])
preds = []

for q in queries_answers.keys():
    response = retrieval_chain.invoke(f"query: {q}")
    preds.append(response)
    print(f"Query: {q}\n")
    print(f"Answer: {response}")
    print(50 * "=" + "\n")

df["pred"] = preds

Query: Какой объём смазки (л) требуется для подшипниковой рамы DY?

Answer: Для подшипниковой рамы DY требуется 0.5 литра смазки (стр. 14).

Query: Укажите кинематическую вязкость рекомендованного масла при 40 °C для роликовых подшипников.

Answer: Кинематическая вязкость рекомендованного масла для роликовых подшипников при 40 °C составляет 150 mm²/s (стр. 14).

Query: Назовите заводскую смазку, рекомендованную для камеры центробежного уплотнения.

Answer: Заводская смазка, рекомендованная для камеры центробежного уплотнения, это FUCHS CENTARUS 4 (стр. 14).

Query: Введите номер телефона сервисной линии для клиентов в Финляндии.

Answer: Номер телефона сервисной линии для клиентов в Финляндии: +358 3 877 350 (стр. 1).

Query: Какова ширина ремня (мм), указанная для диапазона шкивов 170–224 мм (профиль SPB)?

Answer: Ширина ремня для диапазона шкивов 170–224 мм (профиль SPB) составляет 50 мм (стр. 5).

Query: Перечислите три основных требования техники безопасности при подъёме насоса с 

### 4. Оценка качества RAG

Для оценки качества RAG будем использовать BERTScore-F1. В качестве референсных ответов рассмотрим ответы, сгенерированные моделью o3 на основе исходного текстового документа

In [25]:
P, R, F1 = score(df["pred"].tolist(),
                 df["reference"].tolist(),
                 lang="multilingual",
                 verbose=True)

df["P"]   = P.numpy()
df["R"]   = R.numpy()
df["F1"]  = F1.numpy()

calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 3.74 seconds, 3.21 sentences/sec


In [26]:
mean_F1 = df["F1"].mean()
print(f"Mean BERTScore F1: {mean_F1:.4f}")

Mean BERTScore F1: 0.8337


In [27]:
df

,queries,reference,pred,P,R,F1
0,Какой объём смазки (л) требуется для подшипник...,Для подшипниковой рамы типа DY предусмотрен об...,Для подшипниковой рамы DY требуется 0.5 литра ...,0.855604,0.896316,0.875487
1,Укажите кинематическую вязкость рекомендованно...,Для роликовых подшипников рекомендовано масло ...,Кинематическая вязкость рекомендованного масла...,0.836107,0.870289,0.852856
2,"Назовите заводскую смазку, рекомендованную для...",Для смазки камеры центробежного уплотнения сле...,"Заводская смазка, рекомендованная для камеры ц...",0.818384,0.876289,0.846348
3,Введите номер телефона сервисной линии для кли...,Для клиентов в Финляндии действует сервисная л...,Номер телефона сервисной линии для клиентов в ...,0.825234,0.874744,0.849268
4,"Какова ширина ремня (мм), указанная для диапаз...",Для диапазона шкивов 170–224 мм (профиль SPB) ...,Ширина ремня для диапазона шкивов 170–224 мм (...,0.901215,0.937015,0.918766
5,Перечислите три основных требования техники бе...,"1. Поручить операцию компетентному работнику, ...",1. Подъем насоса должен выполняться компетентн...,0.803167,0.860021,0.830623
6,Опишите процедуру проверки уровня масла в подш...,Проверка уровня масла проводится на остановлен...,Для проверки уровня масла в подшипниковом узле...,0.787408,0.766797,0.776966
7,Какой класс NLGI указан для пластичной смазки ...,Класс NLGI для пластичной смазки камеры центро...,Класс по N. L. G. I. для пластичной смазки кам...,0.853442,0.949006,0.898691
8,Какой индекс вязкости DIN‑ISO 2909 указан для ...,Рекомендуемое подшипниковое масло характеризуе...,Индекс вязкости DIN-ISO 2909 для рекомендуемог...,0.818598,0.829331,0.823930
9,На сколько литров объём смазки рамы FFY превыш...,"Объём смазки рамы FFY = 3.0 л, тогда как для р...","Объем смазки рамы FFY составляет 3.0 литра, а ...",0.790716,0.808045,0.799286


### 5. Выводы

В результате работы над тестовым заданием удалось достичь достаточно полных и точных ответов LLM на тестовые вопросы, несмотря на ряд трудностей, которые возникли в процессе разработки.

Стоит отметить следующие моменты:

- **Низкое качество данных**: в качестве исходных данных использовался неоцифрованных pdf-файл. Текстовое содержимое файла удалось получить благодарся использованию OCR. Проблема распознавания таблиц была решена с помощью сторонных сервисов. Однако не всю структуру файла получилось сохранить при получении текстового содержимого, это сказалось на работе Retriever'а.
- **Эксперименты с различными splitter'ами** не выявили преимуществ более продвинутых вариантов в сравнении с `RecursiveCharacterTextSplitter`. Вероятной причиной этого также является низкое качество данных.
- **Продвинутые техники RAG** (HyDE, Multi-Query) позволили увеличить производительность системы и точность ответов, но их использование увеличило время ответа на запрос пользователя.
- **Значение BERTScore F1** является невысоким и составляет ~0.83, хотя LLM дает верные ответы и ссылки на страницы. Возможно, это обусловлено тем, что в качестве референсных рассматривались ответы модели `gpt-o3`, которой в промпт подавался весь документ целиком.

### 6. Возможные улучшения

В качестве основных направлений для улучшения ответов RAG можно выделить следующие:

- **Обработка данных**: Для использования системы в продакшне необходимо усовершенствовать пайплайн обработки данных. Для повышения качества можно использовать для обработки данных более узкоспециализированные модели для работы распознавания таблиц, провеести эксперименты с другими моделями распознавания текста для повышения качества распознавания структуры
- **Продвинутые техники RAG**: Использование таких техник, как SelfQuery, FLARE, реранжирования с помощью CrossEncoder'а и др. Практика показывает, что перечисленные подходы позволяют значительно улучшить качество ответов LLM в продакшне.
- 